In [1]:
import matplotlib.pyplot as plt
import numpy as np
import datasets
import transformers
import re
import torch
import torch.nn.functional as F
from tqdm import tqdm
import random
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import argparse
import datetime
import os
import json
import functools
from multiprocessing.pool import ThreadPool
import time

/opt/conda/envs/detect/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cuda"
dataset = 'writing'
dataset_key = 'document'
n_samples = 100
n_perturbation_list = '1,10,50'
base_model_name = 'facebook/opt-2.7b'
base_model_name_original = 'facebook/opt-2.7b'
mask_filling_model_name = 't5-large'
batch_size = 50
do_top_k = False
top_k = 40
do_top_p = False
top_p = 0.96
output_name = 'n_perturb'
openai_model = None
openai_key = None
mask_top_p = 1.0
cache_dir = '~/.cache'

In [5]:
SEPARATOR = '<<<SEP>>>'
DATASETS = ['writing', 'english', 'german', 'pubmed']


def load_pubmed(cache_dir):
    data = datasets.load_dataset('pubmed_qa', 'pqa_labeled', split='train', cache_dir=cache_dir)
    
    # combine question and long_answer
    data = [f'Question: {q} Answer:{SEPARATOR}{a}' for q, a in zip(data['question'], data['long_answer'])]

    return data

def process_prompt(prompt):
    return prompt.replace('[ WP ]', '').replace('[ OT ]', '')

def process_spaces(story):
    return story.replace(
        ' ,', ',').replace(
        ' .', '.').replace(
        ' ?', '?').replace(
        ' !', '!').replace(
        ' ;', ';').replace(
        ' \'', '\'').replace(
        ' ’ ', '\'').replace(
        ' :', ':').replace(
        '<newline>', '\n').replace(
        '`` ', '"').replace(
        ' \'\'', '"').replace(
        '\'\'', '"').replace(
        '.. ', '... ').replace(
        ' )', ')').replace(
        '( ', '(').replace(
        ' n\'t', 'n\'t').replace(
        ' i ', ' I ').replace(
        ' i\'', ' I\'').replace(
        '\\\'', '\'').replace(
        '\n ', '\n').strip()

def load_writing(cache_dir=None):
    writing_path = 'data/writingPrompts'
    
    with open(f'{writing_path}/valid.wp_source', 'r') as f:
        prompts = f.readlines()
    with open(f'{writing_path}/valid.wp_target', 'r') as f:
        stories = f.readlines()
    
    prompts = [process_prompt(prompt) for prompt in prompts]
    joined = [process_spaces(prompt + " " + story) for prompt, story in zip(prompts, stories)]
    filtered = [story for story in joined if 'nsfw' not in story and 'NSFW' not in story]

    random.seed(0)
    random.shuffle(filtered)

    return filtered

def load_language(language, cache_dir):
    # load either the english or german portion of the wmt16 dataset
    assert language in ['en', 'de']
    d = datasets.load_dataset('wmt16', 'de-en', split='train', cache_dir=cache_dir)
    docs = d['translation']
    desired_language_docs = [d[language] for d in docs]
    lens = [len(d.split()) for d in desired_language_docs]
    sub = [d for d, l in zip(desired_language_docs, lens) if l > 100 and l < 150]
    return sub

def load_german(cache_dir):
    return load_language('de', cache_dir)

def load_english(cache_dir):
    return load_language('en', cache_dir)

def load(name, cache_dir, **kwargs):
    if name in DATASETS:
        load_fn = globals()[f'load_{name}']
        return load_fn(cache_dir=cache_dir, **kwargs)
    else:
        raise ValueError(f'Unknown dataset {name}')

In [6]:
def load_base_model(mask_model, base_model):
    print('MOVING BASE MODEL TO GPU...', end='', flush=True)
    start = time.time()
    try:
        mask_model.cpu()
    except NameError:
        pass
    if openai_model is None:
        base_model.to(DEVICE)
    print(f'DONE ({time.time() - start:.2f}s)')

def load_base_model_and_tokenizer(name):
    if openai_model is None:
        print(f'Loading BASE model {base_model_name}...')
        base_model_kwargs = {}
        if 'gpt-j' in name or 'neox' in name:
            base_model_kwargs.update(dict(torch_dtype=torch.float16))
        if 'gpt-j' in name:
            base_model_kwargs.update(dict(revision='float16'))
        print(name)
        base_model = transformers.AutoModelForCausalLM.from_pretrained(name, **base_model_kwargs, cache_dir=cache_dir)
    else:
        base_model = None

    optional_tok_kwargs = {}
    if "facebook/opt-" in name:
        print("Using non-fast tokenizer for OPT")
        optional_tok_kwargs['fast'] = False
    if dataset in ['pubmed']:
        optional_tok_kwargs['padding_side'] = 'left'
    base_tokenizer = transformers.AutoTokenizer.from_pretrained(name, **optional_tok_kwargs, cache_dir=cache_dir)
    base_tokenizer.pad_token_id = base_tokenizer.eos_token_id

    return base_model, base_tokenizer

# strip newlines from each example; replace one or more newlines with a single space
def strip_newlines(text):
    return ' '.join(text.split())

def generate_data(dataset, key, preproc_tokenizer):
    # load data
    if dataset in DATASETS:
        data = load(dataset, cache_dir)
    else:
        data = datasets.load_dataset(dataset, split='train', cache_dir=cache_dir)[key]
    

    # get unique examples, strip whitespace, and remove newlines
    # then take just the long examples, shuffle, take the first 5,000 to tokenize to save time
    # then take just the examples that are <= 512 tokens (for the mask model)
    # then generate n_samples samples

    # remove duplicates from the data
    data = list(dict.fromkeys(data))  # deterministic, as opposed to set()

    # strip whitespace around each example
    data = [x.strip() for x in data]

    # remove newlines from each example
    data = [strip_newlines(x) for x in data]

    # try to keep only examples with > 250 words
    if dataset in ['writing', 'squad', 'xsum']:
        long_data = [x for x in data if len(x.split()) > 250]
        if len(long_data) > 0:
            data = long_data

    random.seed(0)
    random.shuffle(data)

    data = data[:5_000]

    # keep only examples with <= 512 tokens according to mask_tokenizer
    # this step has the extra effect of removing examples with low-quality/garbage content
    tokenized_data = preproc_tokenizer(data)
    data = [x for x, y in zip(data, tokenized_data["input_ids"]) if len(y) <= 512]

    # print stats about remainining data
    print(f"Total number of samples: {len(data)}")
    print(f"Average number of words: {np.mean([len(x.split()) for x in data])}")

    return data # generate_samples(data[:n_samples], batch_size=batch_size)

In [7]:
API_TOKEN_COUNTER = 0

if openai_model is not None:
    import openai
    assert openai_key is not None, "Must provide OpenAI API key as --openai_key"
    openai.api_key = openai_key

START_DATE = datetime.datetime.now().strftime('%Y-%m-%d')
START_TIME = datetime.datetime.now().strftime('%H-%M-%S-%f')

# define SAVE_FOLDER as the timestamp - base model name - mask filling model name
# create it if it doesn't exist
sampling_string = "top_k" if do_top_k else ("top_p" if do_top_p else "temp")
output_subfolder = f"{output_name}/" if output_name else ""
if openai_model is None:
    base_model_name = base_model_name.replace('/', '_')
else:
    base_model_name = "openai-" + openai_model.replace('/', '_')
SAVE_FOLDER = f"tmp_results/{output_subfolder}{base_model_name}-{mask_filling_model_name}-{sampling_string}/{START_DATE}-{START_TIME}-{dataset}-{n_samples}"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
print(f"Saving results to absolute path: {os.path.abspath(SAVE_FOLDER)}")

mask_filling_model_name = mask_filling_model_name
n_samples = n_samples
batch_size = batch_size
n_perturbation_list = [int(x) for x in n_perturbation_list.split(",")]

cache_dir = cache_dir
os.environ["XDG_CACHE_HOME"] = cache_dir
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
print(f"Using cache dir {cache_dir}")

GPT2_TOKENIZER = transformers.GPT2Tokenizer.from_pretrained('gpt2', cache_dir=cache_dir)

# generic generative model
base_model, base_tokenizer = load_base_model_and_tokenizer(base_model_name_original)

# mask filling t5 model
print(f'Loading mask filling model {mask_filling_model_name}...')
mask_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(mask_filling_model_name, cache_dir=cache_dir)
try:
    n_positions = mask_model.config.n_positions
except AttributeError:
    n_positions = 512

preproc_tokenizer = transformers.AutoTokenizer.from_pretrained('t5-small', model_max_length=512, cache_dir=cache_dir)
mask_tokenizer = transformers.AutoTokenizer.from_pretrained(mask_filling_model_name, model_max_length=n_positions, cache_dir=cache_dir)
if dataset in ['english', 'german']:
    preproc_tokenizer = mask_tokenizer

load_base_model(mask_model, base_model)

print(f'Loading dataset {dataset}...')
data = generate_data(dataset, dataset_key, preproc_tokenizer)

Saving results to absolute path: /home/raghav/cheatGPT/discriminators/uniform-pt/tmp_results/n_perturb/facebook_opt-2.7b-t5-large-temp/2023-03-08-03-29-51-866298-writing-100
Using cache dir ~/.cache
Loading BASE model facebook_opt-2.7b...
facebook/opt-2.7b
Using non-fast tokenizer for OPT
Loading mask filling model t5-large...
MOVING BASE MODEL TO GPU...DONE (2.97s)
Loading dataset writing...


Token indices sequence length is longer than the specified maximum sequence length for this model (706 > 512). Running this sequence through the model will result in indexing errors


Total number of samples: 1011
Average number of words: 309.6142433234421


In [8]:
print(len(data))
data[0]

1011


"Aliens visit earth and are fascinated by other animals but find humans completely unremarkable. Hello, let me begin this report by stating just how thankful I am to have been sent to Earth. It was truly an eye opening experience and without your guidance and funding would have been utterly impossible. Let me give you an overview of how Earth functions, and I think you will be surprised to find that it works very similar to our planet. There exists a large number of organisms going through one stage or another of change while they fall into their respective places on the planet. The conclusion of my research has confirmed what we have believed to be a universal law called net impact theory, which can be described in two parts: 1. a species can only exist if it has a net positive impact on its environment. 2. Species' which detract from their environment will go extinct. My research will hopefully continue to discredit the theory which this universal law replaced - survival of the fitte